# 07. KIỂM ĐỊNH NHÂN QUẢ GRANGER

## Granger Causality Test cho Vietnam

**Mục đích chính:**
- Kiểm tra xem Trade_Openness có "gây ra" (Granger-cause) FDI hay không
- Kiểm tra xem GDP_Growth có "gây ra" FDI hay không
- Xác định độ trễ (lag) tối ưu trong mối quan hệ nhân quả

**Giả thuyết:**
- H0: X không Granger-cause Y (quá khứ của X không giúp dự đoán Y)
- H1: X Granger-causes Y (quá khứ của X có ích trong dự đoán Y)
- Nếu p-value < 0.05 → Bác bỏ H0 → Có nhân quả Granger

**Thông số:**
- maxlag = 2 (kiểm tra độ trễ từ 1 đến 2 năm)
- Test: F-test với p-value
- ⚠️ Giới hạn: Vietnam có 25 observations → maxlag tối đa = 2

**Focus:** Vietnam (25 observations, 2000-2024)

**Lưu ý:** Granger causality ≠ True causality (chỉ là mối quan hệ dự đoán thống kê)

---

In [10]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!")

✅ Libraries imported!


In [11]:
df = pd.read_csv('../01_Data/02_Processed/ASEAN_FDI_GDP_Data_Final.csv')
print(f"✅ Data loaded: {df.shape}")

✅ Data loaded: (225, 8)


## Focus on Vietnam

In [12]:
vietnam = df[df['Country'] == 'Viet Nam'].sort_values('Year')[['Year', 'FDI', 'GDP_Growth', 'Trade_Openness']]
print(f"📊 Vietnam data: {vietnam.shape}")
print(vietnam.head())

📊 Vietnam data: (25, 4)
     Year           FDI  GDP_Growth  Trade_Openness
200  2000  1.298000e+09    6.787316      111.417094
201  2001  1.300000e+09    6.192893      111.955938
202  2002  1.400000e+09    6.320821      116.696869
203  2003  1.450000e+09    6.899063      124.327954
204  2004  1.610000e+09    7.536411      133.016498


## 1. Trade_Openness → FDI

In [13]:
print("\n" + "="*60)
print("GRANGER CAUSALITY: Trade_Openness → FDI")
print("="*60)

data_trade_fdi = vietnam[['FDI', 'Trade_Openness']].dropna()
granger_trade = grangercausalitytests(data_trade_fdi, maxlag=2, verbose=True)

print("\n💡 Interpretation:")
print("If p-value < 0.05 → Trade_Openness CAUSES FDI")
print("If p-value >= 0.05 → NO significant causality")


GRANGER CAUSALITY: Trade_Openness → FDI

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.6600  , p=0.1178  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=3.0400  , p=0.0812  , df=1
likelihood ratio test: chi2=2.8624  , p=0.0907  , df=1
parameter F test:         F=2.6600  , p=0.1178  , df_denom=21, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.8542  , p=0.1853  , df_denom=18, df_num=2
ssr based chi2 test:   chi2=4.7384  , p=0.0936  , df=2
likelihood ratio test: chi2=4.3084  , p=0.1160  , df=2
parameter F test:         F=1.8542  , p=0.1853  , df_denom=18, df_num=2

💡 Interpretation:
If p-value < 0.05 → Trade_Openness CAUSES FDI
If p-value >= 0.05 → NO significant causality


## 2. GDP_Growth → FDI

In [14]:
print("\n" + "="*60)
print("GRANGER CAUSALITY: GDP_Growth → FDI")
print("="*60)

data_gdp_fdi = vietnam[['FDI', 'GDP_Growth']].dropna()
granger_gdp = grangercausalitytests(data_gdp_fdi, maxlag=2, verbose=True)

print("\n💡 Interpretation:")
print("If p-value < 0.05 → GDP_Growth CAUSES FDI")
print("If p-value >= 0.05 → NO significant causality")


GRANGER CAUSALITY: GDP_Growth → FDI

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0186  , p=0.8928  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=0.0213  , p=0.8841  , df=1
likelihood ratio test: chi2=0.0213  , p=0.8841  , df=1
parameter F test:         F=0.0186  , p=0.8928  , df_denom=21, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0089  , p=0.9912  , df_denom=18, df_num=2
ssr based chi2 test:   chi2=0.0227  , p=0.9887  , df=2
likelihood ratio test: chi2=0.0227  , p=0.9887  , df=2
parameter F test:         F=0.0089  , p=0.9912  , df_denom=18, df_num=2

💡 Interpretation:
If p-value < 0.05 → GDP_Growth CAUSES FDI
If p-value >= 0.05 → NO significant causality


## 3. Extract p-values

In [15]:
def extract_pvalues(granger_result):
    pvalues = []
    for lag in range(1, 3):
        test_result = granger_result[lag][0]
        # Extract p-value from F-test
        p_val = test_result['ssr_ftest'][1]
        pvalues.append({'Lag': lag, 'p-value': p_val, 'Significant': 'Yes' if p_val < 0.05 else 'No'})
    return pd.DataFrame(pvalues)

print("\n📊 Trade_Openness → FDI (p-values):")
pval_trade = extract_pvalues(granger_trade)
print(pval_trade)

print("\n📊 GDP_Growth → FDI (p-values):")
pval_gdp = extract_pvalues(granger_gdp)
print(pval_gdp)


📊 Trade_Openness → FDI (p-values):
   Lag   p-value Significant
0    1  0.117806          No
1    2  0.185276          No

📊 GDP_Growth → FDI (p-values):
   Lag   p-value Significant
0    1  0.892790          No
1    2  0.991165          No


## 4. Summary

In [16]:
print("\n" + "="*60)
print("GRANGER CAUSALITY SUMMARY (Vietnam)")
print("="*60)

# Check if any lag is significant
trade_significant = any(pval_trade['p-value'] < 0.05)
gdp_significant = any(pval_gdp['p-value'] < 0.05)

print(f"\n✅ Trade_Openness → FDI: {'YES (significant)' if trade_significant else 'NO (not significant)'}")
print(f"✅ GDP_Growth → FDI: {'YES (significant)' if gdp_significant else 'NO (not significant)'}")


GRANGER CAUSALITY SUMMARY (Vietnam)

✅ Trade_Openness → FDI: NO (not significant)
✅ GDP_Growth → FDI: NO (not significant)


In [18]:
# Save results
with open('../03_Results/Granger_Causality_Results.txt', 'w', encoding='utf-8') as f:
    f.write("GRANGER CAUSALITY TEST (Viet Nam)\n")
    f.write("="*60 + "\n\n")
    
    f.write("1. Trade_Openness -> FDI\n")
    f.write("-"*60 + "\n")
    f.write(pval_trade.to_string() + "\n")
    f.write(f"Conclusion: {'Significant causality' if trade_significant else 'No significant causality'}\n\n")
    
    f.write("2. GDP_Growth -> FDI\n")
    f.write("-"*60 + "\n")
    f.write(pval_gdp.to_string() + "\n")
    f.write(f"Conclusion: {'Significant causality' if gdp_significant else 'No significant causality'}\n")

print("\n✅ Granger Causality results saved!")


✅ Granger Causality results saved!
